In [82]:
import numpy as np
import pandas as pd

In [83]:
df = pd.read_csv('../5-feature-selection/gurgaon_properties_post_feature_selection.csv')

In [84]:
df.head()

,property_type,sector,bedRoom,bathroom,balcony,agePossession,built_up_area,servant room,store room,furnishing_type,luxury_category,floor_category,price
0,0.0,97.0,3,3,3.0,3.0,1310.0,0,0,2,2.0,2.0,1.50
1,0.0,19.0,3,3,2.0,3.0,1500.0,1,0,0,2.0,1.0,0.85
2,0.0,27.0,2,2,1.0,1.0,800.0,0,0,1,1.0,1.0,0.60
3,0.0,119.0,2,1,0.0,1.0,500.0,0,0,0,1.0,0.0,0.30
4,0.0,13.0,3,4,3.0,3.0,1315.0,1,0,2,0.0,2.0,1.55


---



In order to apply Linear Regression we do following 3 task
1. OHE
2.Scalling
3. Log transformation

one hot encode on this column : sector, balcony, agePossession, furnishing type, luxury category, floor category

In [85]:
X = df.drop(columns=['price'])
y = df['price']

In [86]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVR

In [87]:
columns_to_encode = ['sector', 'balcony', 'agePossession', 'furnishing_type', 'luxury_category', 'floor_category']

In [88]:
# Applying the log1p transformation to the target variable
y_transformed = np.log1p(y)

In [89]:
# Creating a column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['property_type', 'bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room']),
        ('cat', OneHotEncoder(drop='first'), columns_to_encode)
    ], 
    remainder='passthrough'
)

In [90]:
# Creating a pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    # ('regressor', LinearRegression())
     ('regressor', SVR(kernel='rbf'))
])

In [91]:
# K-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
try:
    scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')
except ValueError:
    print("An error occurred while calculating the cross-validation score. Please check your data and try again.")

/home/mona/project/real-estate/venv/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:1000: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/mona/project/real-estate/venv/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 139, in __call__
    score = scorer._score(
  File "/home/mona/project/real-estate/venv/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 371, in _score
    y_pred = method_caller(
  File "/home/mona/project/real-estate/venv/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 89, in _cached_call
    result, _ = _get_response_values(
  File "/home/mona/project/real-estate/venv/lib/python3.10/site-packages/sklearn/utils/_response.py", line 239, in _get_response_values
    y_pred, pos_label = prediction_method(X), None
  File "/home/mona/project/real-estate/venv/lib/python3.10/site-packages/sklearn/pi

In [92]:
np.nanmean(scores)

0.8867195085253831

In [93]:
np.nanstd(scores)

0.016639915493863955

In [94]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y_transformed,test_size=0.2,random_state=42)

In [95]:
pipeline.fit(X_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['property_type', 'bedRoom',
                                                   'bathroom', 'built_up_area',
                                                   'servant room',
                                                   'store room']),
                                                 ('cat',
                                                  OneHotEncoder(drop='first'),
                                                  ['sector', 'balcony',
                                                   'agePossession',
                                                   'furnishing_type',
                                                   'luxury_category',
                                                   'floor_category'])])),
                ('regressor', SVR())])

In [96]:
y_pred = pipeline.predict(X_test)

In [97]:
y_pred = np.expm1(y_pred)

In [98]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(np.expm1(y_test),y_pred)

0.4963895911106615

to improve mae we can use different model ,  hyperparameter tunning , feature engineering , feature selection , data cleaning , data preprocessing , data augmentation , data reddecuction , data integration , data transformation , data normalization , data discretization , data encoding , data 